In [1]:
# =======================================================================================
#               SCRIPT "DREAM PIPELINE" V4
# =======================================================================================

# @title Step 1: Install Dependencies
# Keeping essential compatibility fixes for Colab.
print("Phase 1: Installing dependencies...")

# 1. Unsloth for Llama 3
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes -q

# 2. Diffusers and essential packages
!pip install -U diffusers transformers accelerate "invisible-watermark>=0.2.0" safetensors --upgrade -q

print("Dependencies installed. Ready to proceed.\n")


# Essential Imports
import torch
from unsloth import FastLanguageModel
from diffusers import DiffusionPipeline
import time
import os
import gc
import re

# =======================================================================================
#                                 PHASE 1: PROMPT GENERATION
# =======================================================================================

# @title Step 2: Loading Llama 3 8B
print("Phase 2: Loading Llama 3 model...")

max_seq_length = 4096
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/llama-3-8b-Instruct",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)
print("Llama 3 loaded successfully.\n")


# @title Step 3: Sending Prompt to Llama 3
print("Phase 3: Generating prompts...")

messages = [
    {
        "role": "system",
        "content": "You are an expert prompt engineer for image AIs like SDXL. Your mission is to create detailed, vivid, and photorealistic prompts perfect for stock photos. Output ONLY the numbered list of prompts, without any introduction or conclusion. Your prompts should be optimized for SDXL's understanding of natural language."
    },
    {
        "role": "user",
        "content": "Create a list of 5 highly detailed prompts for stock photos. Focus on business, technology, and nature. Include details on lighting, camera settings (like aperture, lens), and mood. Format it as a numbered list."
    },
]

formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

with torch.no_grad():
    inputs = tokenizer([formatted_prompt], return_tensors="pt").to("cuda")
    outputs_tensor = model.generate(**inputs, max_new_tokens=2048, use_cache=True, eos_token_id=tokenizer.eos_token_id)
    text_with_prompts = tokenizer.decode(outputs_tensor[0][len(inputs.input_ids[0]):], skip_special_tokens=True)

print("Prompts generated successfully.\n")
print("--- Generated Prompts ---")
print(text_with_prompts)
print("-------------------------\n")


# @title Step 4: Clearing Memory (VRAM)
print("Phase 4: Unloading Llama 3 to free up memory...")

final_prompts = []
for line in text_with_prompts.strip().split('\n'):
    cleaned_line = re.sub(r'^\d+\.\s*', '', line).strip()
    if cleaned_line:
        final_prompts.append(cleaned_line)

del model, tokenizer, inputs, outputs_tensor
gc.collect()
torch.cuda.empty_cache()

print(f"Memory cleared. {len(final_prompts)} prompts ready.")
print("GPU memory released for SDXL.\n")

# =======================================================================================
#                         PHASE 2: IMAGE GENERATION WITH SDXL
# =======================================================================================

# @title Step 5: Loading SDXL Base and Refiner
print("Phase 5: Loading SDXL Base and Refiner models...")
print("Loading directly to GPU for maximum speed.")

try:
    # Loading BASE model and sending to GPU
    pipe_base = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
        use_safetensors=True
    ).to("cuda")

    # Loading REFINER model and sending to GPU
    pipe_refiner = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-refiner-1.0",
        text_encoder_2=pipe_base.text_encoder_2,
        vae=pipe_base.vae,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16",
    ).to("cuda")

    print("SDXL models loaded to GPU.\n")

    # @title Step 6: Generating Images
    print("Phase 6: Starting image generation loop...")

    output_dir = "generated_images_SDXL_max_speed"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Images will be saved in: '{output_dir}'\n")

    n_steps = 40
    high_noise_frac = 0.8

    for i, prompt in enumerate(final_prompts):
        print(f"Generating image {i+1}/{len(final_prompts)}...")
        print(f"  Prompt: {prompt[:80]}...")

        # Generation is faster as everything is in VRAM
        image = pipe_base(
            prompt=prompt,
            num_inference_steps=n_steps,
            denoising_end=high_noise_frac,
            output_type="latent",
        ).images

        image = pipe_refiner(
            prompt=prompt,
            num_inference_steps=n_steps,
            denoising_start=high_noise_frac,
            image=image,
        ).images[0]

        image_path = os.path.join(output_dir, f"image_SDXL_turbo_{i+1}.png")
        image.save(image_path)
        print(f"  Image saved at: {image_path}\n")

    print("="*60)
    print("Process completed successfully.")
    print(f"Check the '{output_dir}' folder.")

except Exception as e:
    print(f"\nError encountered: {e}")
    print("Ensure you are using a GPU with at least 16GB VRAM (e.g., T4 on Colab Pro or V100).")

Phase 1: Installing dependencies...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 140.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

  Image saved at: generated_images_SDXL_max_speed/image_SDXL_turbo_4.png

Generating image 5/5...
  Prompt: Photograph a stunning sunset over a rugged, rocky coastline, with the warm light...


  0%|          | 0/32 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['high iso ( 3 2 0 0 ) to capture the dynamic play of light and shadow, conveying a sense of awe and wonder.']


  0%|          | 0/8 [00:00<?, ?it/s]

  Image saved at: generated_images_SDXL_max_speed/image_SDXL_turbo_5.png

Process completed successfully.
Check the 'generated_images_SDXL_max_speed' folder.


In [ ]:
#import os, signal
#os.kill(os.getpid(), signal.SIGKILL)
#for gpu vram

In [1]:
!pip install super-image

import os
import torch
from super_image import PanModel, ImageLoader
from PIL import Image
from tqdm import tqdm  # For progress bar visualization

# Settings
input_folder = '/content/generated_images_SDXL_max_speed'  # Change to your folder path
output_folder = '/content/upscaled_folder/'  # Folder to save upscaled images
scale_factor = 4  # 2, 3, or 4 - change as needed
os.makedirs(output_folder, exist_ok=True)  # Creates folder if it doesn't exist

# Load model on GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Running on device: {device}")

model = PanModel.from_pretrained('eugenesiow/pan-bam', scale=scale_factor)
model = model.to(device)  # Move model to GPU
model.eval()  # Inference mode

# List images in the folder (only .jpg and .png)
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.png'))]

# Loop with progress bar
for filename in tqdm(image_files, desc="Upscaling images"):
    input_path = os.path.join(input_folder, filename)
    output_path = os.path.join(output_folder, f"up_{filename}")

    # Load image
    image = Image.open(input_path)

    # Prepare input on GPU
    inputs = ImageLoader.load_image(image).to(device)

    # Inference on GPU
    with torch.no_grad():
        preds = model(inputs)

    # Save (automatically handles moving back to CPU)
    ImageLoader.save_image(preds, output_path)
    print(f"Upscaled: {output_path}")

print("Done! All images upscaled in the output folder.")

Running on device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


https://huggingface.co/eugenesiow/pan-bam/resolve/main/pytorch_model_4x.pt


Upscaling images:  20%|██        | 1/5 [00:03<00:15,  3.91s/it]

Upscaled: /content/upscaled_folder/up_image_SDXL_turbo_2.png


...

Upscaled: /content/upscaled_folder/up_image_SDXL_turbo_3.png


Upscaling images:  80%|████████  | 4/5 [00:13<00:03,  3.42s/it]

Upscaled: /content/upscaled_folder/up_image_SDXL_turbo_4.png


Upscaling images: 100%|██████████| 5/5 [00:16<00:00,  3.21s/it]

Upscaled: /content/upscaled_folder/up_image_SDXL_turbo_1.png
Done! All images upscaled in the output folder.


In [ ]:
# -*- coding: utf-8 -*-
# Version 2.2 - TXT and CSV export support
!pip install transformers==4.51.0
import os
import spacy
import csv
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import torch
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

# Load spaCy model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading spaCy model 'en_core_web_sm' for the first time...")
    spacy.cli.download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

class DreamstimeUploadAutomator:
    def __init__(self, image_folder, categories, num_threads=4):
        self.image_folder = image_folder
        self.categories = categories
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {self.device}")
        self.processor, self.model = self.configure_models()
        self.num_threads = num_threads
        self.model.to(self.device)

    def configure_models(self):
        print("Loading Florence-2-base model...")
        model_id = 'microsoft/Florence-2-base'
        model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval()
        processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
        return processor, model

    def generate_keywords(self, description, max_keywords=50, min_length=3):
        doc = nlp(description.lower())
        keywords = set()
        for token in doc:
            if token.pos_ in ["NOUN", "ADJ", "PROPN"] and not token.is_stop and len(token.text) >= min_length:
                keywords.add(token.text)
        for chunk in doc.noun_chunks:
            clean_chunk = ' '.join(token.text for token in chunk if not token.is_stop and token.pos_ in ["NOUN", "ADJ", "PROPN"])
            if len(clean_chunk) > min_length:
                keywords.add(clean_chunk)
        return ', '.join(list(keywords)[:max_keywords])

    def analyze_image(self, image_path):
        try:
            image = Image.open(image_path).convert("RGB")
            task_prompt = "<MORE_DETAILED_CAPTION>"
            inputs = self.processor(text=task_prompt, images=image, return_tensors="pt").to(self.device)
            generated_ids = self.model.generate(
                input_ids=inputs["input_ids"],
                pixel_values=inputs["pixel_values"],
                max_new_tokens=1024,
                num_beams=3,
                do_sample=False
            )
            generated_text = self.processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
            parsed_answer = self.processor.post_process_generation(generated_text, task=task_prompt, image_size=image.size)
            description = parsed_answer.get(task_prompt, "Could not generate description.")
            title = ' '.join(description.split()[:15])
            keywords = self.generate_keywords(description)

            return {
                'description': description.capitalize(),
                'title': title.capitalize(),
                'keywords': keywords
            }
        except Exception as e:
            print(f"\nError analyzing image {os.path.basename(image_path)}: {e}")
            return None

    def process_image_batch(self, filename):
        full_path = os.path.join(self.image_folder, filename)
        analysis = self.analyze_image(full_path)
        return filename, analysis

    def generate_output_files(self):
        """
        Writes both TXT and CSV files.
        """
        base_name = "dreamstime_upload_florence2_base"
        txt_filename = f"{base_name}.txt"
        csv_filename = f"{base_name}.csv"

        images_to_process = [f for f in os.listdir(self.image_folder)
                               if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        if not images_to_process:
            print("No images found in the folder.")
            return

        print(f"\nFound {len(images_to_process)} images. Processing with Florence-2-base...")

        data_for_csv = []

        with open(txt_filename, "w", encoding="utf-8") as txt_file, ThreadPoolExecutor(max_workers=self.num_threads) as executor:
            future_results = {executor.submit(self.process_image_batch, img): img for img in images_to_process}

            for future in tqdm(future_results, total=len(images_to_process), desc="Analyzing images"):
                filename, analysis = future.result()
                if analysis is None:
                    continue

                # --- Preparing Data ---
                image_data = {
                    'Filename': filename,
                    'Image Name': f"{analysis['title']} generated",
                    'Descriptions': analysis['description'],
                    'Keywords': analysis['keywords'],
                    'Category 1': self.categories[0] if len(self.categories) > 0 else '',
                    'Category 2': self.categories[1] if len(self.categories) > 1 else '',
                    'Category 3': self.categories[2] if len(self.categories) > 2 else '',
                    'Free': 0, 'W-EL': 1, 'P-EL': 1, 'SR-EL': 1, 'SR-Price': 100
                }

                # --- Writing to .txt file ---
                for key, value in image_data.items():
                    txt_file.write(f"{key}: {value}\n")
                txt_file.write("\n") # Separator line

                # --- Storing data for CSV ---
                data_for_csv.append(image_data)

        # --- Creating CSV file ---
        if data_for_csv:
            print("Creating CSV file...")
            headers = data_for_csv[0].keys()

            with open(csv_filename, "w", encoding="utf-8", newline='') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=headers)
                writer.writeheader()
                writer.writerows(data_for_csv)

        print(f"\nFiles created successfully:")
        print(f" -> {txt_filename}")
        print(f" -> {csv_filename}")

def request_image_path():
    while True:
        path = input("Enter the full path to your images folder: ").strip()
        if os.path.isdir(path):
            return path
        else:
            print("Invalid path. Please verify and try again.")

def request_categories():
    categories = []
    print("\nEnter up to 3 categories for your images (e.g., Abstract, Technology, Nature):")
    for i in range(3):
        category = input(f"Category {i+1} (press Enter to skip): ").strip()
        if category:
            categories.append(category)
    return categories

def main():
    print("===========================================")
    print("=== Dreamstime Upload Automation v2.2 ===")
    print("      -- Exports to TXT and CSV --")
    print("===========================================")

    image_folder = request_image_path()
    categories = request_categories()

    while True:
        try:
            num_threads_str = input("\nEnter number of threads (3 or 4 recommended): ")
            num_threads = int(num_threads_str)
            if num_threads > 0:
                break
            else:
                print("Please enter a number greater than zero.")
        except ValueError:
            print("Invalid input. Please enter a number.")

    automator = DreamstimeUploadAutomator(image_folder, categories, num_threads)
    automator.generate_output_files()

if __name__ == "__main__":
    main()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 54.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
